In [2]:
import os
import openai

from getpass import getpass

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [1]:
from langchain.document_loaders import PyPDFLoader

loaders = [
  PyPDFLoader("../05/res/스마트농업_육성사업_추진현황과_개선과제.pdf"),
  PyPDFLoader("../05/res/차세대 한국형 스마트팜 개발.pdf"),
]

docs = []

for loader in loaders:
  docs.extend(loader.load())

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=10
)

splits = text_splitter.split_documents(docs)
print(splits)

[Document(metadata={'source': '../05/res/스마트농업_육성사업_추진현황과_개선과제.pdf', 'page': 0, 'page_label': '1'}, page_content='스마트농업 육성사업 추진현황과 개선과제\n2\n0\n2\n2\n·\n6\n·\n1\n5\n2022. 6. 15.\n스마트농업 육성사업  \n추진현황과 개선과제\n국회예산정책처┃사업평가\nAnalysis on the status and  \nfuture development of  \nSmart Farming projects\n변재연'), Document(metadata={'source': '../05/res/스마트농업_육성사업_추진현황과_개선과제.pdf', 'page': 1, 'page_label': '2'}, page_content='스마트농업 육성사업\n추진현황과 개선과제'), Document(metadata={'source': '../05/res/스마트농업_육성사업_추진현황과_개선과제.pdf', 'page': 2, 'page_label': '3'}, page_content='스마트농업 육성사업 추진현황과 개선과제\n총 괄Ｉ 송병철 예산분석실장\n기획 ․ 조정Ｉ 서세욱 사업평가심의관\n전용수 경제산업사업평가과장\n작 성Ｉ 변재연 경제산업사업평가과 예산분석관\n지 원Ｉ 김창민 경제산업사업평가과 자료분석지원요원\n이채원 경제산업사업평가과 행정실무원\n본 보고서는 ｢국회법｣ 제22조의2 및 ｢국회예산정책처법｣ 제3조에 따라 국회의원의\n의정활동을 지원하기 위하여 발간되었습니다.\n문의:  예산분석실 경제산업사업평가과  | 02) 6788-3777 | eie@nabo.go.kr\n이 책은 국회예산정책처 홈페이지(www.nabo.go.kr)를 통하여 보실 수 있습니다.\n“본 보고서는 담당 분석관의 연구 결과를 바탕으로 작성된 것으로 \n국회예산정책처의 공식의견과는 다를 수 있음을 알려드립니다.”'), Document(metadata={'source': '../05

In [4]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', api_key=os.getenv('OPENAI_API_KEY'))

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_24481/2009276365.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', api_key=os.getenv('OPENAI_API_KEY'))


In [5]:
vectorstore = FAISS.from_documents(splits, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={'k': 13})

In [6]:
llm = ChatOpenAI(model='gpt-4', api_key=os.getenv('OPENAI_API_KEY'))

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_24481/3601464084.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model='gpt-4', api_key=os.getenv('OPENAI_API_KEY'))


In [7]:
from langchain.chains import RetrievalQA

qa1 = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type='stuff',
  retriever=retriever,
)

query = "한국형 스마트팜이란?"
print(qa1.run(query=query))

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_24481/4274906481.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(qa1.run(query=query))


한국형 스마트팜은 사물인터넷, 빅데이터, 인공지능, 로봇 등을 활용하여 농산물의 생육환경을 최적상태로 관리하고 노동력 절감과 생산성 향상을 구현하는 효율적인 농업형태입니다. 이는 고령화와 높은 노동강도로 인해 청년인구의 유입이 어려운 현재의 우리나라 농업 구조에 대한 대안으로 제시되고 있습니다. 한국형 스마트팜 기술개발 프로젝트는 4기관 19개 전담부서가 협업하여 핵심 요소 및 원천 기반기술의 확보를 위해 연구 역량을 집중하고 있으며, 국내 농업여건에 적합하게 기술수준별로 스마트팜 모델을 3가지 단계로 구분하여 개발을 추진하고 있습니다. 이 단계별 스마트팜은 1세대(편리성 증진), 2세대(생산성 향상-네덜란드추격형), 3세대(글로벌산업화-플랜트 수출형)로 구분되고 있습니다.


In [8]:
os.environ['COHERE_API_KEY'] = ''

compressor = CohereRerank()

compression_retriever = ContextualCompressionRetriever(
  base_compressor=compressor,
  base_retriever=retriever,
)

compressed_docs = compression_retriever.get_relevant_documents(query)
print(compressed_docs)

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_24481/961778741.py:3: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank()
/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_24481/961778741.py:10: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  compressed_docs = compression_retriever.get_relevant_documents(query)


[Document(metadata={'source': '../05/res/차세대 한국형 스마트팜 개발.pdf', 'page': 0, 'page_label': '1', 'relevance_score': 0.99792516}, page_content='차세대\xa0한국형스마트팜\xa0개발\n국립농업과학원\xa0농업공학부\xa0스마트팜개발과\xa0이현동\xa0연구관\n한국형\xa0스마트팜\xa0개발\xa0프로젝트\xa0!\n우리나라\xa0농업은\xa0고령화와\xa0높은\xa0노동강도로\xa0인해\xa0청년인구의\xa0유입이\xa0어려운\xa0구조로\xa0타\xa0분야보다\xa0생산인구\xa0절벽화가\xa0더욱\n가시화되고\xa0있다.\xa0이에\xa0대한\xa0대안으로\xa0제시되고\xa0있는\xa0스마트팜은\xa0사물인터넷,\xa0빅데이터,\xa0인공지능,\xa0로봇\xa0등을\xa0활용하여\n농산물의\xa0생육환경을\xa0최적상태로\xa0관리하고\xa0노동력\xa0절감과\xa0생산성\xa0향상을\xa0구현하는\xa0효율적인\xa0농업형태이다.\xa0\n차세대\xa0한국형\xa0스마트팜\xa0기술개발\xa0프로젝트는\xa04기관\xa019개\xa0전담부서가\xa0협업하여\xa0핵심\xa0요소\xa0및\xa0원천\xa0기반기술의\xa0확보를\xa0위해\n연구\xa0역량을\xa0집중하고\xa0있고\xa0국내\xa0농업여건에\xa0적합하게\xa0기술수준별로\xa0스마트팜\xa0모델을\xa03가지\xa0단계로\xa0구분하여\xa0개발을\n추진하고\xa0있다.\xa0단계별\xa0스마트팜은\xa01세대(편리성\xa0증진),\xa02세대(생산성\xa0향상-네덜란드추격형),\xa03세대(글로벌산업화-플랜트\n수출형)로\xa0구분되고\xa0기술의\xa0단계적\xa0개발과\xa0실용화\xa0계획을\xa0통해\xa0노동력과\xa0농자재의\xa0사용을\xa0줄이고,\xa0생산성과\xa0품질을\n제고함으로\xa0농가소득과\xa0연계하며,\xa0나아가\xa0영농현장의\xa0애로와\xa0연

In [9]:
qa2 = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type='stuff',
  retriever=compression_retriever,
)

print(qa2.run(query=query))

한국형 스마트팜은 사물인터넷, 빅데이터, 인공지능, 로봇 등을 활용하여 농산물의 생육환경을 최적상태로 관리하고 노동력 절감과 생산성 향상을 구현하는 효율적인 농업형태를 말합니다. 이는 외국의 선진 기술을 그대로 수입하거나 단순 모방하지 않고, 우리 농업 환경과 현장 상황에 적합한 기술을 개발하여 농업선진국과 경쟁할 수 있는 독자적인 농업생산기술을 확보하는 것을 목표로 합니다. 차세대 한국형 스마트팜 기술개발 프로젝트는 1세대(편리성 증진), 2세대(생산성 향상), 3세대(글로벌산업화)로 구분하여 개발을 추진하고 있습니다.
